# FOF策略

In [1]:
import os, sys, argparse, logging

# 设置logging格式和级别
logging.basicConfig(
    level=logging.DEBUG,
    format='%(filename)s-line%(lineno)d %(levelname)s: %(message)s'
    )

from datetime import datetime

# 首先将framework的目录加进sys.path中，方便使用相对路径导入
framework_path = os.path.join(os.path.abspath(''), '../framework')
if framework_path not in sys.path:
    sys.path.append(framework_path)

this_path = os.path.abspath('')
if this_path not in sys.path:
    sys.path.append(this_path)

from import_func import getSvc

raw_data_svc = getSvc('LxwWinddbRawDataSvc')
date_svc = getSvc('DateSvc')
date_svc.setTradeDays(raw_data_svc.getTradeDays())
constant_svc = getSvc('ConstantSvc')


init YamlSvc
init YamlSvc


utils.py-line160 INFO: NumExpr defaulting to 8 threads.
lxw_winddb_sql_raw_data_svc.py-line69 DEBUG: raw_data_svc.sqlQuery: SELECT TRADE_DAYS FROM asharecalendar WHERE S_INFO_EXCHMARKET='SSE' ORDER BY TRADE_DAYS


init LxwWinddbRawDataSvc
init MysqlDbConnectorSvc
init DateSvc


date_svc.py-line31 INFO: DateSvc: set trade days


init ConstantSvc


## 1. 测试单期fof策略

In [2]:
# 债券基金筛选条件
bond_filter_condition = {
    'fund_age':3*12,                # 基金年龄 >= n 个月  
    'fund_del': -1e-8, # 基金停牌时间 < 0
    'netasset_total':4e8,    # 基金规模 >= n 元
    'f_prt_stocktotot': -30            # 过去4个季度，股票平均仓位 <= 30
}

# 股票基金筛选条件
stock_filter_condition = {
    'fund_age':3*12,                # 基金年龄 >= n 个月  
    'fund_del': -1e-8, # 基金停牌时间 < 0
    'netasset_total':2e8,    # 基金规模 >= n 元
    'f_prt_stocktotot': 60,           # 过去4个季度，股票平均仓位 >= 60
    'is_indexfund': -0.5   # 不是指数基金
}

filter_conditions = {
    'bond_conditions': bond_filter_condition, 
    'stock_conditions': stock_filter_condition,
}

In [3]:
from test_fof_strategy import TestFofStrategy

# test
tfs = TestFofStrategy('tfs')
tfs.getDataset().print()

tfs.setArgsForAlgs({
    'select_fund_condition': filter_conditions
})
bond_fund_style, stock_fund_style = tfs.run(datetime(2016,12,31))

lxw_winddb_sql_raw_data_svc.py-line69 DEBUG: raw_data_svc.sqlQuery: SELECT TRADE_DAYS FROM asharecalendar WHERE S_INFO_EXCHMARKET='SSE' ORDER BY TRADE_DAYS
date_svc.py-line31 INFO: DateSvc: set trade days
alg_base.py-line19 INFO: alg tfs: init args
alg_base.py-line19 INFO: alg merge_fund_raw_data: init args
merge_fund_raw_data.py-line34 INFO: loading raw data
merge_fund_raw_data.py-line41 DEBUG: loading chinamutualfunddescription
lxw_winddb_local_raw_data_svc.py-line51 DEBUG: loading chinamutualfunddescription from local_db
merge_fund_raw_data.py-line41 DEBUG: loading chinamutualfundassetportfolio
lxw_winddb_local_raw_data_svc.py-line51 DEBUG: loading chinamutualfundassetportfolio from local_db
merge_fund_raw_data.py-line41 DEBUG: loading chinamutualfundnav
lxw_winddb_local_raw_data_svc.py-line51 DEBUG: loading chinamutualfundnav from local_db
merge_fund_raw_data.py-line41 DEBUG: loading chinamutualfundsector
lxw_winddb_local_raw_data_svc.py-line51 DEBUG: loading chinamutualfundsector 

group: root
	asset: cash
	group: bond
	group: stock


date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWithIndex: some index not found in data.index
date_svc.py-line51 DEBUG: DateSvc.cutDataWi

In [6]:
print('bond_fund_style: ')
print(bond_fund_style.head())
print(bond_fund_style.shape)

print('*' * 50)
print('stock_fund_style: ')
print(stock_fund_style.head())
print(stock_fund_style.shape)

bond_fund_style: 
                default
000503.OF  conservative
550004.OF  conservative
000143.OF  conservative
202101.OF        active
550018.OF        active
(32, 1)
**************************************************
stock_fund_style: 
          value_growth   industry market_size
160131.SZ        VALUE        NaN       LARGE
519152.OF       GROWTH     GROWTH       SMALL
000244.OF        VALUE  FINANCIAL       LARGE
000054.OF       GROWTH     GROWTH         NaN
000143.OF       GROWTH   CONSUMER       SMALL
(47, 3)
